<a href="https://colab.research.google.com/github/sumants-dev/CIS545-Project/blob/main/Seasonal_weather_data__analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIS 545 - Weather Big Data Analysis

# 0 Introduction

* Team members:  Ashley Chang, Dana Yang, Sumant Shringari

**Our thesis: **

> Predict weather with different indicators of air pollution



**Our approach**

For this set of data we experimented with three different models to evulate the best result. 
1.  Naive Bayes classification with random forest 
2.  Time series with Arima 
3.  Times series with Sarima 

**Our data set consist of:** 

  Air pollution at city level 

    * Date range: 2020 - 2021
    * Final data set is 
  Air pollution By Country

    * Date range: 2010 - 2017
    * Final dataset is 1352 records



**Overview of results **


> Indented block



#1 Data setup

### 1.0 Importing data and library

In [1]:
!pip install pandasql
!pip install pandas
!pip install requests
!pip install lxml
!pip install nltk
!pip install pandasql

In [2]:
# Imports
import pandas as pd
import pandasql as psql
from lxml import html
import requests
import nltk


In [4]:
!pip install matplotlib
!pip install seaborn

In [3]:
import numpy as np 
import json
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import datetime
from datetime import timedelta
import glob
import seaborn as sns
import re
import os

In [4]:
air_polution_pd = pd.read_csv("./Data/raw/PM2.5 Global Air Pollution 2010-2017.csv")

Importing 2021 data

In [5]:
acqin_data_pd21 = pd.read_csv("./Data/raw/waqi-covid19-airqualitydata-2020.csv", error_bad_lines=False, header = 0, skiprows=4)

/Users/ashleyfchang/opt/anaconda3/envs/cis545/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


Importing 2020 data

In [7]:
acqin_data_pd20_Q1 = pd.read_csv("./Data/raw/waqi-covid19-airqualitydata-2020Q1.csv", error_bad_lines=False, header = 0, skiprows=4)
acqin_data_pd20_Q2 = pd.read_csv("./Data/raw/waqi-covid19-airqualitydata-2020Q2.csv", error_bad_lines=False, header = 0, skiprows=4)
acqin_data_pd20_Q3 = pd.read_csv("./Data/raw/Data/waqi-covid19-airqualitydata-2020Q3.csv", error_bad_lines=False, header = 0, skiprows=4)
acqin_data_pd20_Q4 = pd.read_csv("./Data/raw/waqi-covid19-airqualitydata-2020Q4.csv", error_bad_lines=False, header = 0, skiprows=4)

FileNotFoundError: [Errno 2] No such file or directory: './Data/raw/waqi-covid19-airqualitydata-2020Q2.csv'

Importing 2019 data 

In [ ]:
acqin_data_pd19_Q1 = pd.read_csv("./Data/raw/Data/waqi-covid19-airqualitydata-2019Q1.csv", error_bad_lines=False, header = 0, skiprows=4)
acqin_data_pd19_Q2 = pd.read_csv("./Data/raw/Data/waqi-covid19-airqualitydata-2019Q2.csv", error_bad_lines=False, header = 0, skiprows=4)
acqin_data_pd19_Q3 = pd.read_csv("./Data/raw/Data/waqi-covid19-airqualitydata-2019Q3.csv", error_bad_lines=False, header = 0, skiprows=4)
acqin_data_pd19_Q4 = pd.read_csv("./Data/raw/aqi-covid19-airqualitydata-2019Q4.csv", error_bad_lines=False, header = 0, skiprows=4)

In [ ]:
population = pd.read_csv("./Data/raw/Data//city_population.csv")

## 1.2 ACQIN air polution dataset data cleaning and wrangling

Data sourced from 
*  Polluation and tempature data from AQICN : https://aqicn.org/data-platform/covid19/verify/821c0eaf-e7cd-41e7-a6e3-84b8dfea1988
      * Raw data:
            *  Time line: 2019 - 2021 
            *  # of records : 613,561 after transpose
            *  # of attributes: 21
* Population data from world population :  https://worldpopulationreview.com/world-cities
      * Raw data:
            *  Time line: 2020 - 2021 
            *  # of records : 1170
            *  # of attributes: 3
Overall goals of data cleaning includes: 


*  Combine datasets between each quaters and year for the acqin dataset
*  Drop any observeration that had 0 count in data collected for the sample 
*  Normalize the population dataset and join it into the ACQIN dataset

### 1.1.1 Data overview

Let's first take a quick look at the format of the dataset

In [ ]:
acqin_data_pd20_Q1.head(5)

A quick look at our data set and the features 

In [ ]:
num_country = psql.sqldf("SELECT COUNT(DISTINCT Country) AS num_country FROM acqin_data_pd20_Q1")
num_city = psql.sqldf("SELECT COUNT(DISTINCT city) AS num_city FROM acqin_data_pd20_Q1")
features = psql.sqldf("SELECT DISTINCT Specie AS features FROM acqin_data_pd20_Q1")
print(num_city)
print(num_country)
print(features)

### 1.1.2 Data cleaning and joining

Lets join the four quarters of 2020 with 2021 of dataset together 


In [ ]:

frames = [acqin_data_pd20_Q1, acqin_data_pd20_Q2,acqin_data_pd20_Q3,acqin_data_pd20_Q4, acqin_data_pd21, acqin_data_pd19_Q1, acqin_data_pd19_Q2,acqin_data_pd19_Q3,acqin_data_pd19_Q4]
acqin_all_pollution_df = pd.concat(frames)


Drop where the count is 0; drop records where there are no samples collected meaning the count is 0. 

In [ ]:
acqin_all_pollution_valid_values = acqin_all_pollution_df[(acqin_all_pollution_df[['count']] != 0).all(axis =1)]

In [ ]:
print(acqin_all_pollution_valid_values.shape[0])

Next, lets transpose the dataset so we have the features as columns

In [ ]:
acqin_all_pollution_df_transposed = acqin_all_pollution_valid_values.pivot_table(index= ['Date','Country', 'City'], columns='Specie', values='median', fill_value=0).rename_axis(None, axis=1).reset_index()
acqin_all_pollution_df_transposed['Date'] = acqin_all_pollution_df_transposed['Date'].astype(str)

In [ ]:
acqin_all_pollution_df_transposed = acqin_all_pollution_df_transposed.sort_values(by=['City', 'Date'])
acqin_all_pollution_df_transposed

In [ ]:
query = '''
SELECT *
FROM acqin_all_pollution_df_transposed
WHERE  dew != 0.0 AND pressure != 0.0 AND so2 != 0.0  AND pm25 != 0.0
AND temperature != 0.0
'''

vaild_values_pollution = psql.sqldf(query)

vaild_values_pollution

In [ ]:
print(vaild_values_pollution.shape[0])

In [ ]:
query = '''
SELECT City, Date, temperature 
FROM acqin_all_pollution_df_transposed
WHERE temperature = 0.0
ORDER BY City, Date 

'''

temp_data = psql.sqldf(query)
temp_data

Drop where tempature is 0 or NULL

In [ ]:
#acqin_all_pollution_df_transposed = acqin_all_pollution_df_transposed[(acqin_all_pollution_df_transposed[['temperature']] != 0).all(axis =1)]

In [ ]:
print(acqin_all_pollution_df_transposed.shape[0])

Let's add a two Year and month field for joining into the population data later

In [ ]:
acqin_all_pollution_df_transposed['short_date'] = acqin_all_pollution_df_transposed['Date'].str[:7]

In [ ]:
vaild_values_pollution['short_date'] = vaild_values_pollution['Date'].str[:7]
vaild_values_pollution['month'] = vaild_values_pollution['Date'].str[5:7]

In [ ]:
vaild_values_pollution['datetime'] = pd.to_datetime(df['Date'], format = '%Y%m%d')

Next we want to normalize the population data across the 12 months time period to match with the acqin data set

In [ ]:
print(population.shape[0])

In [ ]:
## normailize the population acorss the 12 months in the year for each city 
query19 = "SELECT Name, Prev -(Population - Prev)- (Population - Prev) as StartPopulation, (Population - Prev)/12  as increment FROM population "
query20 = "SELECT Name, Prev -(Population - Prev) as StartPopulation, (Population - Prev)/12  as increment FROM population "
query21 = "SELECT Name,  Prev  as StartPopulation, (Population - Prev)/12  as increment FROM population"

## defining the 2020 and 2021 dates

m2019 = {'Date': ['2019-01', '2019-02','2019-03','2019-04', 
                  '2019-05','2019-06-01','2019-07-01', '2019-08-01',
                  '2019-09','2019-10', '2019-11','2019-12'], 'num': [1,2,3,4,5,6,7,8,9,10,11,12]}
m2020 = {'Date': ['2020-01', '2020-02','2020-03','2020-04', 
                  '2020-05','2020-06-01','2020-07-01', '2020-08-01',
                  '2020-09','2020-10', '2020-11','2020-12'], 'num': [1,2,3,4,5,6,7,8,9,10,11,12]}
m2021 = {'Date': ['2021-01', '2021-02','2021-03','2021-04', 
                  '2021-05','2021-06','2021-07', '2021-08',
                  '2021-09','2021-10', '2021-11','2021-12'], 'num': [1,2,3,4,5,6,7,8,9,10,11,12]}
months19 = pd.DataFrame(data = m2019, columns =['Date', 'num'])
months20 = pd.DataFrame(data = m2020, columns =['Date', 'num'])
months21 = pd.DataFrame(data = m2021, columns =['Date', 'num'])

distributed_population19 = psql.sqldf(query19)
distributed_population20 = psql.sqldf(query20)
distributed_population21 = psql.sqldf(query21)

#cross join the get the set of each city with each month 
query_month19 = 'SELECT * FROM months19 CROSS JOIN distributed_population19 ORDER BY Name'
distributed_population19 = psql.sqldf(query_month19)

query_month20 = 'SELECT * FROM months20 CROSS JOIN distributed_population20 ORDER BY Name'
distributed_population20 = psql.sqldf(query_month20)

query_month21 = 'SELECT * FROM months21 CROSS JOIN distributed_population21 ORDER BY Name'
distributed_population21 = psql.sqldf(query_month21)

#calculate the rolling total 
rolling_pop_query19 = '''
SELECT Date, Name, StartPopulation + increment*num as Population, num as month 
FROM distributed_population19
'''
distributed_population19 = psql.sqldf(rolling_pop_query19)
rolling_pop_query20 = '''
SELECT Date, Name, StartPopulation + increment*num as Population, num as month 
FROM distributed_population20
'''
distributed_population20 = psql.sqldf(rolling_pop_query20)

rolling_pop_query21 = '''
SELECT Date, Name, StartPopulation + increment*num as Population, num as month 
FROM distributed_population21
'''
distributed_population21 = psql.sqldf(rolling_pop_query21)

frames = [distributed_population20, distributed_population21,distributed_population19]
distributed_population = pd.concat(frames)

In [ ]:
distributed_population


Let's join the population dataset 

In [ ]:
query = '''
        SELECT * 
        FROM acqin_all_pollution_df_transposed a LEFT JOIN distributed_population p 
        ON (a.CITY = p.Name ) 
        ''' 

  #AND a.Date LIKE CONCAT(p.Date+'%') 
  #a.CITY = p.Name AND

copyOfpop = distributed_population.copy(deep = True)
weather_with_pop = pd.merge(acqin_all_pollution_df_transposed, distributed_population,how='inner', left_on=['City', 'short_date'], right_on=['Name', 'Date'])

weather_with_pop_small = pd.merge(vaild_values_pollution, copyOfpop ,how='inner', left_on=['City', 'short_date'], right_on=['Name', 'Date'])
#weather_with_pop = psql.sqldf(query)

In [ ]:
print(weather_with_pop_small.shape[0])

In [ ]:
weather_with_pop.head(5)

In [ ]:
print(acqin_all_pollution_df_transposed.shape[0])

In [ ]:
print(weather_with_pop.shape[0])

## 1.1.3 drop extra columns and rename

In [ ]:
weather_with_pop = weather_with_pop.drop(columns=['Date_y', 'Name'])
weather_with_pop= weather_with_pop.rename(columns={'Date_x': 'Date'})

In [ ]:
weather_with_pop_small = weather_with_pop_small.drop(columns=['Date_y', 'Name'])
weather_with_pop_small= weather_with_pop_small.rename(columns={'Date_x': 'Date'})

In [ ]:
weather_with_pop

In [ ]:
weather_with_pop_small['datetime'] = pd.to_datetime(df['Date'], format = '%Y%m%d')

Finally, we will export our cleanned dataframe out to CSV

In [ ]:
weather_with_pop['datetime'] = pd.to_datetime(df['Date'], format = '%Y%m%d')

In [ ]:
weather_with_pop.to_csv('/content/drive/MyDrive/MCIT/CIS545/Data/weather_with_pop.csv')

##1.3 Country level pollution data

### 1.3.1 Extraction of standardlize country codes

For each of our data set, we extract the country name to country code mapping through wikipedia. Our approach is to make a request to the wikipedia page, and then use xpath to find the list of country names and country code.

In [ ]:
def get_country_codes():
    wiki = requests.get("https://en.wikipedia.org/wiki/ISO_3166-1_alpha-3")
    dom_tree = html.fromstring(wiki.content)
    xpath = "//div[@class=\"plainlist\"]/ul/li"
    country_list  = dom_tree.xpath(xpath)
    country_map = {}
    for country_elem in country_list:
        country_map[country_elem[2].text] = country_elem[1].text
    return country_map
        
def set_country_value(df):
    country_to_code = get_country_codes()
    #df["Country"] = df['Country Name'].apply(lambda elem: country_to_code.get(elem))
    df.insert(0, 'iso_code',df['Country Name'].apply(lambda elem: country_to_code.get(elem)) )
    return df

In [ ]:
air_polution_pd = set_country_value(air_polution_pd)


Overall goals of data cleaning includes: 


*   Extract a standardlized date range and group the data by year and month 
*   Ensure consist country data format by updating country to include ISO ; https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes (3-alpha code)
*   Drop columns that are not needed for analysis 
*   Drop any rows that have nulls 
*   Export cleaned dataset




First, we will print the initial pollution dataframe to understand the shape of the data we are working with. 

In [ ]:
air_polution_pd.head(5)

The next step we will use Pandas describe function to understand our dataset a bit further. 


*   The data set consist of 240 countries data in air pollution between 2010 to 2017 
*   There are no country missing a data record between 2010 to 2017, since 2010 - 2017 all have 240 count of records 



In [ ]:
air_polution_pd.describe()

In [ ]:
air_polution_pd_cleaned = air_polution_pd.dropna()

air_polution_pd_cleaned = air_polution_pd_cleaned.drop(columns=['Country Name', 'Country Code'])
air_polution_pd_cleaned.head(5)

In [ ]:
 air_polution_pd_cleaned_transposed =  air_polution_pd_cleaned.set_index('iso_code').stack().reset_index().rename(columns={'level_1': 'year', 0:'percent'})

In [ ]:
air_polution_pd_cleaned_transposed.head(5)

Next, we wil reformat the dataset that is consistent with the other datasets. We will have the iso_code, year and percent as columns of our dataframe

Finally, we will export our cleanned dataframe out to CSV

In [ ]:
air_polution_pd_cleaned_transposed.to_csv('/content/drive/MyDrive/MCIT/CIS545/Data/pollution_data_cleaned.csv')

# 2 Graphing trends and analyzing the data set



*   Line graphs on trend: overall change with year on Y axis 
*   Histogram for top 10 countries: comparison between each country by the attribute 
*   Globe view heat map
*   Feature correlations heatmap
*   Trend: % change by year group by region 

Below are graphing functions

In [ ]:
def linePlot(dataset):
  sns.set(rc = {'figure.figsize':(15,8)})
  sns.lineplot(data=dataset, x="year", y="percent", hue="iso_code", style="iso_code")
  plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  

Annual year pollution of top 10 countries 

In [ ]:


query_a = '''
SELECT year, percent, iso_code
FROM air_polution_pd_cleaned_transposed
WHERE  iso_code IN(
  SELECT DISTINCT iso_code 
  FROM air_polution_pd_cleaned_transposed 
  ORDER BY percent DESC
  LIMIT 20
)
ORDER BY year ASC, iso_code, percent
'''

graph_query0 = psql.sqldf(query_a)
linePlot(graph_query0)
plt.title('Countries with highest pollution')

In [ ]:

query_b = '''
SELECT year, percent, iso_code
FROM air_polution_pd_cleaned_transposed
WHERE  iso_code IN(
  SELECT DISTINCT iso_code 
  FROM air_polution_pd_cleaned_transposed 
  ORDER BY percent 
  LIMIT 20
)
ORDER BY year ASC, iso_code, percent
'''

graph_query0 = psql.sqldf(query_b)
linePlot(graph_query0)
plt.title('Countries with lowest pollution')

Worst polluters at the city level

In [ ]:
def cityLevelBarPlot(data_input):
  sns.set_theme(style="whitegrid")
  sns.set(rc = {'figure.figsize':(25,8)})
  sns.barplot(x ="City", y = "pollution", data=data_input )

In [ ]:
query1 = '''
SELECT  City, max(pm25) as pollution
FROM weather_with_pop
WHERE Date LIKE '2021-01%'
GROUP BY City
ORDER BY pm25 DESC
LIMIT 20
'''

query2 = '''
SELECT City, max(pm25) as pollution
FROM weather_with_pop
WHERE Date LIKE '2021-11%'
GROUP BY City
ORDER BY pm25 DESC
LIMIT 20
'''


graph_query1 = psql.sqldf(query1)
graph_query2 = psql.sqldf(query2)


In [ ]:
query3 = '''
SELECT City, max(pm25) as pollution, short_date as date
FROM weather_with_pop
WHERE City IN (
  SELECT City
  FROM weather_with_pop
  WHERE Date LIKE '2021-06%' 
  GROUP BY City
  ORDER BY pm25 DESC
  LIMIT 10
)
GROUP BY City, short_date
ORDER BY short_date, pm25 DESC 
'''

graph_query3 = psql.sqldf(query3)

In [ ]:
print(graph_query3.shape[0])

In [ ]:
graph_query1

In [ ]:
cityLevelBarPlot(graph_query1)
plt.title('Top pollution cities in the summer (Jun)')

In [ ]:
cityLevelBarPlot(graph_query2)
plt.title('Top pollution cities in the winter (Nov)')

Generally, we are seeing that there is a rise in pm 2.5 in the winter seasons between November to Feburary 

In [ ]:
sns.set(rc = {'figure.figsize':(30,8)})
sns.lineplot(data=graph_query3, x="date", y="pollution", hue="City", style="City")

training time 
accuracy 
notablity of the approach 


Graph relationship between pollution and tempature 

In [7]:
query = '''
SELECT AVG(temperature) as temperature , AVG(pm25) as pm25, short_date
FROM vaild_values_pollution
GROUP BY short_date 
'''
temp_and_pm25 = psql.sqldf(query)


PandaSQLException: (sqlite3.OperationalError) no such table: vaild_values_pollution
[SQL: 
SELECT AVG(temperature) as temperature , AVG(pm25) as pm25, short_date
FROM vaild_values_pollution
GROUP BY short_date 
]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

Observations


*   The winter month when the tempature are lower, there is a higher rate of pollution. The immediate relationship between the pollution and tempature is inverse relationship
*   The is a gradual decrease in pollutions 
*   There is a gradual increase in highest and lowest tempature 



In [8]:
import matplotlib.pyplot as plt
Date =  temp_and_pm25['short_date']
temp = temp_and_pm25['temperature']
pm25 = temp_and_pm25['pm25']

plt.plot(Date, temp, label = "temperature" )
plt.plot(Date, pm25, label = "pollution" )
plt.legend()

plt.title("Temp and pollution comparision plot", fontsize = 20) # for title
plt.xlabel("Date", fontsize = 12) # label for x-axis
plt.ylabel("Temp /pollution", fontsize = 12) # label for y-axis
plt.show()

NameError: name 'temp_and_pm25' is not defined

# 3 Creating Model

#### **3.1.** Correlation of Feature Variables

There will be two sets of each steps because the same process was ran acorss one dataset with population and one with out population information. Both datasets have about 250k records. The dataset without population information also had none deterministic values like 0.0 dropped from the dataset.

In [9]:
weather_with_pop_corr = weather_with_pop.corr()
sns.heatmap(data = weather_with_pop_corr, annot=False)

NameError: name 'weather_with_pop' is not defined

dew, pressure, no2, population, wind-gust, uvi, so2, wind-speed, pm25

In [10]:
vaild_values_pollution = vaild_values_pollution.drop(columns = ['mepaqi'])

NameError: name 'vaild_values_pollution' is not defined

In [ ]:
weather_corr = vaild_values_pollution.corr()
sns.heatmap(data = weather_corr, annot=False)

#### **3.2** Select the key columns

In [ ]:
new_weather_df = weather_with_pop[['dew', 'pressure', 'no2', 'Population', 'wind-gust', 'so2', 'wind-speed', 'pm25', 'pm10', 'temperature','City','Country','month', 'humidity', 'short_date', 'co']]

In [ ]:
new_weather_df2 = vaild_values_pollution[['dew', 'pressure', 'no2', 'wind-gust', 'so2', 'pm25', 'pm10', 'temperature','City','Country','month', 'humidity', 'short_date','co']]

#### **3.2** Encode Categorical Variables using LabelEncoder

In [ ]:
from sklearn.preprocessing import LabelEncoder

el = LabelEncoder()
city_labels = el.fit_transform(new_weather_df['City'])
country_labels = el.fit_transform(new_weather_df['Country'])
short_date_labels = el.fit_transform(new_weather_df['short_date'])

city_labels2 = el.fit_transform(new_weather_df2['City'])
country_labels2 = el.fit_transform(new_weather_df2['Country'])
short_date_labels2 = el.fit_transform(new_weather_df2['short_date'])


In [ ]:
new_weather_df['city_labels'] = city_labels
new_weather_df['country_labels'] = country_labels
new_weather_df['short_date_labels'] = short_date_labels


new_weather_df2['city_labels'] = city_labels2
new_weather_df2['country_labels'] = country_labels2
new_weather_df2['short_date_labels'] = short_date_labels2

In [ ]:
# updated correlation heatmap based on feature variables
updated_weather_df_corr = new_weather_df.corr()

#  heatmap for easier visualization
sns.heatmap(data = updated_weather_df_corr, annot=False)


In [ ]:
updated_weather_df2_cor = new_weather_df2.corr()
sns.heatmap(data = updated_weather_df2_cor, annot=False)

#### **3.4** Split into Features and Label

In [ ]:
features = new_weather_df[['dew', 'pressure', 'no2', 'Population', 'wind-gust', 'so2', 'wind-speed', 'pm25', 'short_date_labels', 'country_labels', 'city_labels', 'pm10','co' ]]
labels = new_weather_df['temperature']

In [ ]:
print(features.shape[0])

In [ ]:
features2 = new_weather_df2[['dew', 'pressure', 'no2', 'wind-gust', 'so2', 'pm25', 'pm10', 'temperature','month', 'humidity', 'country_labels', 'city_labels', 'short_date_labels','co']]
labels2 = new_weather_df2['temperature']


In [ ]:
print(features2.shape[0])

Change all data type to int for using Naive Bayes classification

In [ ]:
features = features.astype('int64')
labels = labels.astype('int64')

In [ ]:
features2 = features2.astype('int64')
labels2 = labels2.astype('int64')

### **3.5** Modelling

#### **3.5.1** Split Data into Train and Test

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.20, random_state=42)

In [ ]:
# split by years 
#Training 2019 
#Validation 2020 
#Testing 2021



In [ ]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(features2, labels2, test_size=0.20, random_state=42)

#### **3.5.2**  Naive Bayes classification 

In [ ]:
from sklearn.naive_bayes import GaussianNB


# TODO model 
nb = GaussianNB()
nb.fit(x_train, y_train)
y_pred = nb.predict(x_test)
y_pred

In [ ]:
nb2 = GaussianNB()
nb2.fit(x_train2, y_train2)
y_pred2 = nb2.predict(x_test2)
y_pred2

In [ ]:
comparison =  np.where(y_pred == y_test, True, False)
naive_bayes_acc = sum(comparison== True)/ len(y_test)

print(naive_bayes_acc)

In [ ]:
comparison2 =  np.where(y_pred2 == y_test2, True, False)
naive_bayes_acc2 = sum(comparison2== True)/ len(y_test2)

print(naive_bayes_acc2)

Plot the comparision 

#### **3.5.3** Dimensionality Reduction with PCA

In [ ]:
y_pred2
prediction = pd.DataFrame(y_pred2)

In [ ]:
y_test2
test = pd.DataFrame(y_test2)


In [ ]:
test['month'] = x_test2['month']

In [ ]:
test = test.reset_index()
test = test.rename(columns={'temperature': 'actual'})

In [ ]:
test = pd.merge(test, prediction, left_index=True, right_index= True)

In [ ]:
test = test.rename(columns={0: 'prediction'})
test

In [ ]:

import plotly.express as px

px.line(test, x = "month", y ="actual")

In [ ]:
# TO DO PCA
# scaling
# find num components to use
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_x_train = scaler.fit_transform(x_train)
scaled_x_test = scaler.transform(x_test)

pca = PCA(n_components = 12)
pca_x_train = pca.fit_transform(scaled_x_train)


print(pca_x_train.shape)

print(pca.components_)
# TO DO plot for explained variance
pca.explained_variance_ratio_
plt.plot(np.arange(0, 12), np.cumsum(pca.explained_variance_ratio_))
plt.plot(np.arange(0, 12), [0.95]*12)
plt.show()

In [ ]:
scaler2 = StandardScaler()
scaled_x_train2 = scaler.fit_transform(x_train2)
scaled_x_test2 = scaler.transform(x_test2)

pca = PCA(n_components = 14)
pca_x_train2 = pca.fit_transform(scaled_x_train2)


print(pca_x_train.shape)

print(pca.components_)
# TO DO plot for explained variance
pca.explained_variance_ratio_
plt.plot(np.arange(0, 14), np.cumsum(pca.explained_variance_ratio_))
plt.plot(np.arange(0, 14), [0.95]*14)
plt.show()

In [ ]:
explained_variance = pca.explained_variance_ratio_
print(explained_variance)

In [ ]:
pca = PCA(n_components=11 ) # TODO update this number
x_train_pca = pca.fit_transform(scaled_x_train)
x_test_pca  = pca.transform(scaled_x_test)
x_train = x_train_pca
x_test = x_test_pca

In [ ]:
pca = PCA(n_components=11 ) # TODO update this number
x_train_pca2 = pca.fit_transform(scaled_x_train2)
x_test_pca2  = pca.transform(scaled_x_test2)
x_train2 = x_train_pca2
x_test2 = x_test_pca2

Reducing the PCA to 2 and plotting the result to see the outcome 

In [ ]:
# visualize 2 components of PCA
pca_plot = PCA(n_components=2)
comp = pca_plot.fit_transform(x_train_pca)
data = np.transpose(np.array(comp))
plt.scatter(data[0][0:5000], data[1][0:5000])

In [ ]:
# visualize 2 components of PCA
pca_plot = PCA(n_components=2)
comp = pca_plot.fit_transform(x_train_pca2)
data = np.transpose(np.array(comp))
plt.scatter(data[0][0:5000], data[1][0:5000])

#### 3.5.4 Random Forest

In [ ]:
def print_best_params(results):
    print('best_params: {}\n'.format(results.best_params_))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

#from sklearn.grid_search import GridSearchCV



tree = RandomForestClassifier()

parameters = {'max_depth':[1, 10], 'n_estimators': [1,10]}
rf_cv = GridSearchCV(tree,parameters).fit(x_train, y_train)

In [ ]:
#use this 
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
X, y = make_regression(n_features=4, n_informative=2,
                       random_state=0, shuffle=False)

regr = RandomForestRegressor(max_depth=2, random_state=0, n_jobs =2, verbose =1)
regr.fit(X, y)

print(regr.predict([[0, 0, 0, 0]]))

In [ ]:
#from sklearn.grid_search import GridSearchCV

tree2 = RandomForestClassifier()

parameters2 = {'max_depth':[1, 10], 'n_estimators': [1,10]}
rf_cv2 = GridSearchCV(tree2,parameters2).fit(x_train2, y_train2)

In [ ]:
print_best_params(rf_cv)

In [ ]:
# TODO
from sklearn.metrics import accuracy_score

rf_cv.fit(x_train, y_train)
prediction = rf_cv.predict(x_test)
test_accuracy = accuracy_score(prediction, y_test)
print(test_accuracy)

In [ ]:
rf_cv2.fit(x_train2, y_train2)
prediction2 = rf_cv2.predict(x_test2)
test_accuracy2 = accuracy_score(prediction2, y_test2)
print(test_accuracy2)

## 3.6 Modeling with Timeseries approach (Arima and Sarima 

### 3.6.0 Arima time series model 

In [ ]:
%%capture
!pip install pandas
!pip install plotly-express
!pip install statsmodels
!pip install tqdm

In [ ]:
 
import pandas as pd
from sklearn.model_selection import train_test_split
import statsmodels as sm 
from sklearn.metrics import r2_score
from tqdm import tqdm
import plotly.express as px
from sklearn.preprocessing import StandardScaler
# model imports
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm1
from sklearn.metrics import mean_squared_error, r2_score
import itertools
import math
from sklearn.decomposition import PCA
import pickle


### 3.6.1 Scaling Data By Standard Scalar

In [ ]:
def get_country_df(country):
    df = pd.read_csv('../Data/global_average_yearly_temp_with_features_clean.csv')
    df = df[df["iso_code"] == country]
    df["Year_idx"] = pd.to_datetime(df.year, format="%Y")
    df = df.set_index("Year_idx")
    df.index = df.index.to_period("Y")
    return df

def get_test_train_valid(country, train_split = '1950', validation_split = '1980'):
    country_df = get_country_df(country)
    
    train = country_df.loc[:train_split]
    valid = country_df.loc[train_split: validation_split]
    test  = country_df.loc[validation_split:]
    
    return train, test, valid

def scale(train_x, valid_x, test_x):
    scaler = StandardScaler()
    model_x = pd.concat([train_x, valid_x])

    scaler.fit(model_x)
    model_x = scaler.transform(model_x)
    train_x = scaler.transform(train_x)
    valid_x = scaler.transform(valid_x)
    test_x  = scaler.transform(test_x)
    return train_x, valid_x, test_x, model_x

def pca_fitter(train_x, threshold = .99):
    n_features = train_x.shape[1]
    
    for n in range(1, n_features + 1):
        pca_model = PCA(n)
        pca_model.fit(train_x)
        
        if sum(pca_model.explained_variance_ratio_) >= threshold:
            break
    return pca_model


### 3.6.2 Hypertuning Model Parameters With Training and Validation Set

In [ ]:
def eval_arima_excog_walk_forward_validation_model(train_x, train_y, test_x, test_y, arima_order):
    # DO NOT USE THIS 

    history = [x for x in train_y]
    predictions = []
    
    for i in range(len(test_x)):
        model = ARIMA(history, exog = train_x, order = arima_order)
        model_fit = model.fit()
        
        y_hat = model_fit.forecast(exog = test_x.iloc[[i]])
        predictions.append(y_hat[0])
        
        train_x = pd.concat([train_x, test_x.iloc[[i]]], axis = 0)
        history.append(test_y[i])

    rmse = math.sqrt(mean_squared_error(test_y, predictions))
    return model_fit, rmse, predictions

def eval_arima_excog(train_x, train_y, test_x, test_y, arima_order, trend = 'c'):
    # DO NOT USE THIS 
    model = ARIMA(train_y, exog = train_x, order = arima_order, trend = trend)
    model_fit = model.fit()
    predictions = model_fit.forecast(len(test_x), exog = test_x)
    rmse = math.sqrt(mean_squared_error(test_y, predictions))
    return model_fit, rmse, predictions

def eval_sarimax_excog(train_x, train_y, test_x, test_y, arima_order):
    # USE THIS
    model = sm1.tsa.statespace.SARIMAX(train_y, exog = train_x, order = arima_order, \
                                       time_varying_regression = True, mle_regression = False, measurement_error = True)
    model_fit = model.fit(disp = 0)
    predictions = model_fit.forecast(len(test_x), exog = test_x)
    rmse = math.sqrt(mean_squared_error(test_y, predictions))
    predictions.index = test_y.index
    return model_fit, rmse, predictions

def eval_excog_models(train_x, train_y, test_x, test_y, p_values, d_values, q_values, f):
    arima_orders = itertools.product(*[p_values, d_values, q_values])
    arima_orders = list(arima_orders)
    best_order, best_score = None, float("inf") 
    
    count = 0
    for arima_order in tqdm(arima_orders):
        try:
            _, rmse, _ = f(train_x, train_y, test_x, test_y, arima_order)
            
            if rmse < best_score:
                best_score, best_order = rmse, arima_order
                print(f"ARIMA RMSE = {best_score}")
        except Exception as e:
            print(e)
            pass
        count += 1
    print('DONE')
    return best_order

In [ ]:
def ml(country, p_values = [0, 1, 2, 3], d_values = range(0, 3), q_values = range(0, 3)):
    train, test, valid = get_test_train_valid(country)

    train_x, train_y = train[train.columns[2:-2]], train["AvgYearlyTemp"]
    test_x, test_y = test[test.columns[2:-2]], test["AvgYearlyTemp"]
    valid_x, valid_y = valid[valid.columns[2:-2]], valid["AvgYearlyTemp"]

    train_x, valid_x, test_x, model_x = scale(train_x, valid_x, test_x)

    pca_model = pca_fitter(train_x)
    train_x = pca_model.transform(train_x)
    valid_x = pca_model.transform(valid_x) 
    test_x  = pca_model.transform(test_x)
    model_x = pca_model.transform(model_x)
    
    model_y = pd.concat([train_y, valid_y])

    best = eval_excog_models(train_x, train_y, valid_x, valid_y, p_values, d_values, q_values, eval_sarimax_excog)

    model_fit, rmse, predictions = eval_sarimax_excog(model_x, model_y, test_x, test_y, best)
    
    return best, test_y, predictions

In [ ]:
def plot(df, country, layout, line1 = 'AvgYearlyTemp', line2 = 'Predictions', x = 'year', error = 'AvgTempUncertainty'):
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go


    fig = go.Figure(layout = layout)
    trace_1 = go.Line(name = f"Actual {country} Avg Yearly Temp", 
                     x = test[x],
                     y = test[line1],
                     error_y = dict(type='data', array = test[error]))


    trace_2 = go.Line(name = f"Predictions of {country} Avg Yearly Temp", 
                     x = test[x],
                     y = test[line2])

    fig.add_trace(trace_1)
    fig.add_trace(trace_2)
    
    return fig